# Introduction
- A callback is a powerful tool to customize the behavior of Keras models during training, evaluation, or inference.
    - Examples include `tf.keras.callbacks.TensorBoard` to visualize training process and results with TensorBoard, or `tf.keras.callbacks.ModelCheckpoint` to periodically save your model during training.

# Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

# Keras callbacks overview
- All callbacks subclass the `keras.callbacks.Callback` class and override a set of methods called at various stages of training, testing, and predicting. 
- Callbacks are useful to get a view on internal states and statistics of the model during training.
- You can pass a list of callbacks (as the keyword argument callbacks) to the following model methods:
    - `keras.Model.fit()`
    - `keras.Model.evaluate()`
    - `keras.Model.predict()`

# An overview of callback methods
## Global methods
- `on_(train|test|predict)_begin(self, logs=None)`
    - Called at the beginning of `fit`/`evaluate`/`predict`
- `on_(train|test|predict)_end(self, logs=None)`
    - Called at the end of `fit`/`evaluate`/`predict`

## Bacth-level methods for training/testing/predicting
- `on_(train|test|predict)_batch_begin(self, batch, logs=None)`
    - Called right before processing a batch during training/testing/predicting
- `on_(train|test|predict)_end(self, logs=None)`
    - Called at the end of training/testing/predicting a batch
    - Within this method, `logs` is a dict containing the metrics results
    
## Epoch-level methods (training only)
- `on_epoch_begin(self, epoch, logs=None)`
    - Called at the beginning of an epoch during training
- `on_epoch_end(self, epoch, logs=None)`
    - Called at the end of an epoch during training

# A basic example
- To get started, let's import TensorFlow and define a simple Sequential model.

In [2]:
# Define the Keras model to add callbacks to
def get_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(1, input_dim=784))
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )
    return model

- Then, load the MNIST data for training and testing from Keras datasets API.

In [3]:
# Load example MNIST data and pre-process it
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape(-1, 784).astype("float32") / 255.0
X_test = X_test.reshape(-1, 784).astype("float32") / 255.0

# Limit the data to 1000 samples
X_train = X_train[:1000]
y_train = y_train[:1000]
X_test = X_test[:1000]
y_test = y_test[:1000]

- Now, define a simple custom callback that logs when:
    - `fit`|`evaluate`|`predict` starts and ends
    - Each epoch starts and ends
    - Each training batch starts and ends
    - Each evaluation (test) batch starts and ends
    - Each inference (prediction) batch starts and ends

In [4]:
class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))

    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop training; got log keys: {}".format(keys))

    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start testing; got log keys: {}".format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop testing; got log keys: {}".format(keys))

    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start predicting; got log keys: {}".format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop predicting; got log keys: {}".format(keys))

    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: start of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: end of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: start of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))

- Let's try it out.

In [5]:
model = get_model()

In [6]:
# model.fit(
#     X_train,
#     y_train,
#     batch_size=128,
#     epochs=1,
#     verbose=0,
#     validation_split=0.5,
#     callbacks=[CustomCallback()]
# )

# Usage of `logs` dict
- The `logs` dict contains the loss value and all the metrics at the end of a batch or epoch.

In [7]:
class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_test_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_epoch_end(self, epoch, logs=None):
        print(
            "The average loss for epoch {} is {:7.2f} "
            "and mean absolute error is {:7.2f}.".format(
                epoch, logs["loss"], logs["mean_absolute_error"]
            )
        )

In [8]:
model = get_model()

In [9]:
model.fit(X_train, y_train, batch_size=128, epochs=2, verbose=0, callbacks=[LossAndErrorPrintingCallback()])

For batch 0, loss is   26.18.
For batch 1, loss is  970.32.
For batch 2, loss is   18.06.
For batch 3, loss is    7.09.
For batch 4, loss is    8.33.
For batch 5, loss is    6.48.
For batch 6, loss is    5.59.
For batch 7, loss is    5.53.
The average loss for epoch 0 is  133.96 and mean absolute error is    5.98.
For batch 0, loss is    4.47.
For batch 1, loss is    4.64.
For batch 2, loss is    3.90.
For batch 3, loss is    4.35.
For batch 4, loss is    3.42.
For batch 5, loss is    5.11.
For batch 6, loss is    4.51.
For batch 7, loss is    4.10.
The average loss for epoch 1 is    4.32 and mean absolute error is    1.69.


# Usage of `self.model` attribute
- In addition to receiving log information when one of their methods is called, callbacks have access to the model associated with the current round of training/evaluation/inference: `self.model`.
- Here are of few of the things you can do with self.model in a callback:
    - Set `self.model.stop_training = True` to immediately interrupt training
    - Mutate hyperparameters of the optimizer (available as `self.model.optimizer`), such as `self.model.optimizer.learning_rate`
    - Save the model at period intervals
    - Record the output of `model.predict()` on a few test samples at the end of each epoch, to use as a sanity check during training
    - Extract visualizations of intermediate features at the end of each epoch, to monitor what the model is learning over time
    - etc.

# Example of Keras callback applications
## Early stopping at minimum loss
- This first example shows the creation of a Callback that stops training when the minimum of loss has been reached, by setting the attribute `self.model.stop_training (boolean)`. 
- Optionally, you can provide an argument `patience` to specify how many epochs we should wait before stopping after having reached a local minimum.
- `tf.keras.callbacks.EarlyStopping` provides a more complete and general implementation.

In [10]:
import numpy as np

In [11]:
class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
    # Stop training when the loss is at its minimum, i.e. the loss stops decreasing
    
    def __init__(self, patience=0):
        super().__init__()
        self.patience = patience
        
        # best_weights to store the weights at which the minim loss occurs
        self.best_weights = None
        
    def on_train_begin(self, logs=None):
        # The number of epoch it had waited when loss is no longer minimum
        self.wait = 0
        
        # The epoch the training stops at
        self.stopped_epoch = 0
        
        # Initialize the best as infinity
        self.best = np.Inf
        
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            
            # Record the best weight if current results is better
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch")
                self.model.set_weights(self.best_weights)
                
    def on_train_end(self, logs=None):
        if self.stopped_epoch >= 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))

In [12]:
model = get_model()
model.fit(X_train, y_train, batch_size=64, steps_per_epoch=5, epochs=30, verbose=0, callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()])

For batch 0, loss is   34.09.
For batch 1, loss is  779.00.
For batch 2, loss is   34.57.
For batch 3, loss is   12.96.
For batch 4, loss is    8.38.
The average loss for epoch 0 is  173.80 and mean absolute error is    8.36.
For batch 0, loss is    7.86.
For batch 1, loss is    5.13.
For batch 2, loss is    5.26.
For batch 3, loss is    5.65.
For batch 4, loss is    4.97.
The average loss for epoch 1 is    5.77 and mean absolute error is    1.97.
For batch 0, loss is    5.15.
For batch 1, loss is    4.42.
For batch 2, loss is    5.58.
For batch 3, loss is    5.40.
For batch 4, loss is    6.67.
The average loss for epoch 2 is    5.44 and mean absolute error is    1.89.
For batch 0, loss is   10.62.
For batch 1, loss is   23.47.
For batch 2, loss is   43.63.
For batch 3, loss is   80.44.
For batch 4, loss is   95.27.
The average loss for epoch 3 is   53.93 and mean absolute error is    6.35.
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping


## Learning rate scheduling
- In this example, we show how a custom Callback can be used to dynamically change the learning rate of the optimizer during the course of training.
- See `callbacks.LearningRateScheduler` for more general implementations.

In [13]:
class CustomLearningRateScheduler(keras.callbacks.Callback):
    # Learning rate scheduler sets the learning rate according to schedule.
    
    def __init__(self, schedule):
        super().__init__()
        self.schedule = schedule
        
    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
            
        # Get the current learning rate from model's optimizer
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        
        # Call schedule function to get the scheduled learning rate
        scheduled_lr = self.schedule(epoch, lr)
        
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))

In [14]:
LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05),
    (6, 0.01),
    (9, 0.005),
    (12, 0.001)
]

In [15]:
def lr_schedule(epoch, lr):
    # Helper function to retrieve the scheduled learning rate based on epoch
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr

In [16]:
model = get_model()

model.fit(X_train, y_train, batch_size=64, steps_per_epoch=5, epochs=15, verbose=0, callbacks=[LossAndErrorPrintingCallback(), CustomLearningRateScheduler(lr_schedule)])


Epoch 00000: Learning rate is 0.1000.
For batch 0, loss is   22.86.
For batch 1, loss is  936.21.
For batch 2, loss is   27.80.
For batch 3, loss is   10.42.
For batch 4, loss is    5.87.
The average loss for epoch 0 is  200.63 and mean absolute error is    8.34.

Epoch 00001: Learning rate is 0.1000.
For batch 0, loss is    7.46.
For batch 1, loss is    5.55.
For batch 2, loss is    7.00.
For batch 3, loss is    4.31.
For batch 4, loss is    5.58.
The average loss for epoch 1 is    5.98 and mean absolute error is    2.00.

Epoch 00002: Learning rate is 0.1000.
For batch 0, loss is    4.82.
For batch 1, loss is    3.56.
For batch 2, loss is    5.08.
For batch 3, loss is    3.56.
For batch 4, loss is    4.61.
The average loss for epoch 2 is    4.33 and mean absolute error is    1.66.

Epoch 00003: Learning rate is 0.0500.
For batch 0, loss is    4.41.
For batch 1, loss is    4.06.
For batch 2, loss is    4.39.
For batch 3, loss is    3.71.
For batch 4, loss is    3.32.
The average loss

# Built-in Keras callbacks
- Be sure to check out the existing Keras callbacks by reading the API docs.
- Applications include logging to CSV, saving the model, visualizing metrics in TensorBoard, and a lot more!